# Day 2: Optimization Fundamentals and Mosek
## Advanced Portfolio Optimization with Transaction Costs and Leverage Constraints
This notebook demonstrates state-of-the-art portfolio optimization using Mosek, incorporating realistic constraints for large-scale, multi-asset strategies.

In [ ]:
from mosek.fusion import *
import numpy as np
import pandas as pd

def advanced_portfolio_optimization(returns: pd.DataFrame, initial_weights: np.ndarray, target_return: float, transaction_cost: float, leverage_limit: float) -> np.ndarray:
    n = returns.shape[1]
    mu = returns.mean().values
    Sigma = returns.cov().values

    with Model("AdvancedPortfolio") as M:
        w = M.variable("w", n, Domain.greaterThan(0.0))  # No shorting
        t = M.variable("t", n, Domain.unbounded())  # Transaction cost variable

        # Objective: Minimize variance + transaction costs
        M.objective("obj", ObjectiveSense.Minimize, Expr.add(Expr.dot(w, Expr.mul(Sigma, w)), Expr.mul(transaction_cost, Expr.sum(t))))

        # Constraints
        M.constraint("return", Expr.dot(mu, w), Domain.greaterThan(target_return))
        M.constraint("leverage", Expr.sum(w), Domain.lessThan(leverage_limit))
        M.constraint("transaction", Expr.sub(w, initial_weights), Domain.equalsTo(t))

        M.solve()
        return w.level()

# Simulate data for a large portfolio
np.random.seed(42)
n_assets = 100  # Scalable to thousands
n_days = 252
prices = pd.DataFrame(np.random.normal(loc=100, scale=10, size=(n_days + 1, n_assets)), columns=[f'Asset{i+1}' for i in range(n_assets)])
returns = prices.pct_change().dropna()

# Parameters
initial_weights = np.ones(n_assets) / n_assets
target_return = 0.15
transaction_cost = 0.001  # 10 bps
leverage_limit = 1.5  # Max leverage

# Optimize
optimal_weights = advanced_portfolio_optimization(returns, initial_weights, target_return, transaction_cost, leverage_limit)
print('Optimal Portfolio Weights (first 5 assets):')
print(optimal_weights[:5])

## Notes
- **Mosek Fusion API**: Efficiently handles large-scale optimization problems.
- **Transaction Costs**: Modeled using absolute value constraints, approximated via SOCP.
- **Leverage Constraints**: Enforce regulatory and risk limits, critical for funds operating under strict mandates.